Reference: [Build a Retrieval Augmented Generation (RAG) App](https://python.langchain.com/v0.2/docs/tutorials/rag/)

In [1]:
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv() # invoke langsmith

llm = ChatOpenAI(model="gpt-3.5-turbo")

## Build vector database

https://how.wtf/how-to-use-chroma-db-step-by-step-guide.html

In [3]:
import os

from langchain import hub
from langchain_community.document_loaders import UnstructuredMarkdownLoader, PyPDFLoader
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embedding_cache_path = "embedding_cache"
db_collection_names = ["CNS16190-zh_TW_only_provision", "ts_103701v010101p_without_content"]
document_root_path = "docs"
documents = ["CNS16190-zh_TW_only_provision.pdf", "ts_103701v010101p_without_content.pdf"]
# 創建一個空列表來保存所有分割後的文檔
all_splits = []

# 加載和分割每個文檔
for idx, document in enumerate(documents):
    if document.startswith("CNS16190"):
        collection_name = "CNS16190"
    elif document.startswith("ts_103701"):
        collection_name = "TS103701"


    if os.path.isdir(os.path.join(embedding_cache_path, collection_name)):
        # database already exists, load it
        print("Loading existing database")
        if collection_name == "CNS16190":
            vectorstore_CNS16190 = Qdrant.from_existing_collection(
            embedding=OpenAIEmbeddings(),
            path=embedding_cache_path + '/' + collection_name,
            collection_name=collection_name,
            )
            retriever_CNS16190 = vectorstore_CNS16190.as_retriever()

        elif collection_name == "TS103701":
            vectorstore_TS103701 = Qdrant.from_existing_collection(
            embedding=OpenAIEmbeddings(),
            path=embedding_cache_path + '/' + collection_name,
            collection_name=collection_name,
            )
            retriever_TS103701 = vectorstore_TS103701.as_retriever()
   
    else:
        # database does not exist, create it
        print("Creating new database")
        print(f"Processing document: {document}")


        if document.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(document_root_path, document))
        elif document.endswith(".md"):
            loader = UnstructuredMarkdownLoader(os.path.join(document_root_path, document), mode="elements")
        else:
            continue
        doc = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        splits = text_splitter.split_documents(doc)
        # 將所有分割後的文檔添加到同一個 Qdrant 集合中
        if collection_name == "CNS16190":
            vectorstore_CNS16190 = Qdrant.from_documents(
                splits,
                embedding=OpenAIEmbeddings(),
                path=embedding_cache_path + '/' + collection_name,
                collection_name=collection_name,
            )
            # 設置檢索器
            retriever_CNS16190 = vectorstore_CNS16190.as_retriever()
        elif collection_name == "TS103701":
            vectorstore_TS103701 = Qdrant.from_documents(
                splits,
                embedding=OpenAIEmbeddings(),
                path=embedding_cache_path + '/' + collection_name,
                collection_name=collection_name,
            )
            # 設置檢索器
            retriever_TS103701 = vectorstore_TS103701.as_retriever()

print("All documents have been processed and added to the Qdrant collection.")


Loading existing database
Loading existing database
All documents have been processed and added to the Qdrant collection.


In [4]:
prompt = hub.pull(
    "rlm/rag-prompt"
)  # https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=07050030-99d2-5925-9ec6-8a8fb67164d3

prompt.messages[0].prompt.template="你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案："
prompt



ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案：'))])

## Build rag chain

In [5]:
import pandas as pd


# concat all docs as a string
def format_docs(docs):
    retrieval_docs = "\n\n".join(doc.page_content for doc in docs)

    result = retrieval_docs
    print(result)
    return result

df = pd.read_csv("docs/test_data_from_GoogleSheet.csv")
test_data = "\n\n測試資料範例：\n\n"
for idx in range(3):
    row = df.iloc[idx]
    test_data += f"控制措施{row['provision']}，detail:{row['detail']}，question:{row['Test Scenario']}，answer:{row['answer']}\n\n"




In [ ]:
# from operator import itemgetter
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate

# prompt_CNS16190 = ChatPromptTemplate.from_template(f"{query}\n\n根據控制措施{row['provision']}和{row['detail']}\
#                                             在CNS16190中找出控制措施{row['provision']}到下一個控制措施之間的詳細內容。\
#                                             輸出格式為: 控制措施{row['provision']}: 控制措施的詳細內容")
# prompt_TS103701 = ChatPromptTemplate.from_template(f"{query}\n\n在ETSI TS 103 701中找出{row['provision']}Test group{row['provision']}到下一個Test group之間的內容，\
#                           HINT: 控制措施就是provision。\
#                           輸出格式為: Test group{row['provision']}內容: ")
# prompt_testScenario = ChatPromptTemplate.from_template(f"{query}\n\n根據控制措施和detail在ETSI TS 103 701中找出與控制措施最相關Test group，\
#                           並生成測試流程來供測試者驗證answer 是否真的符合控制措施detail要求。")

# result_CNS16190 = (
#     {"context_CNS126190": retriever_CNS16190 | format_docs, "question": RunnablePassthrough()}
#     | prompt_CNS16190
#     | llm
# )

# result_TS103701 = (
#     {"context_TS103701": retriever_TS103701 | format_docs, "question": RunnablePassthrough()}
#     | prompt_TS103701
#     | llm
# )
# testScenario_generator = (
#     {"attribute": RunnablePassthrough()} | prompt_testScenario | llm
# )

# question_testScenario = (
#     testScenario_generator | {"result_CNS16190": result_CNS16190, "result_TS103701": result_TS103701, "attribute": RunnablePassthrough()} 
#     | itemgetter("result_CNS16190", "result_TS103701")
#     | prompt_testScenario
# )

In [28]:
from langchain_core.runnables import RunnablePassthrough

rag_chain_CNS16190 = (
    {"context": retriever_CNS16190 | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)
rag_chain_TS103701 = (
    {"context": retriever_TS103701 | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

# test_scenarios_responder = 
# This is equivalent to following without using format_docs():
# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# the path of the rag_chain:
# 1. get "question" from rag_chain.invoke()
# 2. search the most relevant context from the retriever
# 3. concat the docs from retriever in format_docs() as a string
# 4. format the string as a prompt
# 5. query the prompt to the llm

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


## Ask llm

In [177]:
df = pd.read_csv("docs/test_data_from_GoogleSheet.csv")

df

,provision,status,support,detail,Test Scenario,answer,reason,result
0,5.1-1,MC,NaN,如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的...,問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置...,使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或...,NaN,NaN
1,5.1-3,M,NaN,用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。,與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通...,本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TL...,NaN,NaN
2,5.2-1,M,NaN,製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資...,漏洞揭露政策明確規定了安全研究人員和其他人報告問題的流程。 此類政策可以根據需要進行更新，以...,通過D-Link 網站（https://support.dlink.com/ReportVu...,NaN,NaN
3,5.3-13,M,NaN,製造商應以對使用者易於理解、透明的方式公開定義的支援期限。,消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。,NaN,NaN
4,5.3-16,M,NaN,消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。,問題：1. 使用者如何驗證設備型號名稱（透過設備上的標籤或透過實體介面）？,此產品的型號標籤上有相關的生產編號資訊。,NaN,NaN
5,5.4-1,M,NaN,持久存儲中的敏感安全參數應由設備安全的存儲。,安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉...,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。,NaN,NaN
6,5.4-3,M,NaN,關鍵安全參數不得硬編碼於使用設備軟體原始碼中。,設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證...,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...,NaN,NaN
7,5.4-4,M,NaN,用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個...,為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果...,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\n管理...,NaN,NaN
8,5.5-1,M,NaN,消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。,安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN
9,5.5-5,M,NaN,設備功能只有在認證存取之後允許透過網路介面進行配置安全相關修改。例外情況是設備所依賴的網路服...,注意：例外的協定包括 ARP、DHCP、DNS、ICMP 和 NTP。範例：與安全性相關的變...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN


### One row testing

In [12]:
row = df.iloc[8]
# query = f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
query = f"控制措施{row['provision']}，detail:{row['detail']}"
print(
    f"控制措施 {row['provision']}\n\n【detail】\n{row['detail']}\n\n【question】\n{row['Test Scenario']}\n\n【answer】\n{row['answer']}"
)

print(f"\n【Test scenario】\n{row['Test Scenario']}")


控制措施 5.5-1

【detail】
消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。

【question】
安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，很難提供有關密碼學或其他安全措施的規範性建議，而不會面臨此類建議很快就會過時的風險。問題：1. 消費者物聯網設備使用什麼最佳實踐加密技術來安全通訊？

【answer】
D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密方式進行，並採用最高等級的Cipher 符合FIPS 14--3 的安全強度模組進行，並且會隨時注意最先安全通報，檢查是否現行加密模組是否被發現漏洞，需要時即時更新。

【Test scenario】
安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，很難提供有關密碼學或其他安全措施的規範性建議，而不會面臨此類建議很快就會過時的風險。問題：1. 消費者物聯網設備使用什麼最佳實踐加密技術來安全通訊？


In [18]:
# rag_chain.invoke(f"{query}\n\n根據控制措施和detail在ETSI TS 103 701中找出與控制措施最相關Test group，\
#                  並生成測試流程來供測試者驗證answer 是否真的符合控制措施detail要求。")
# rag_chain_CNS16190.invoke(f"{query}\n\n根據控制措施和detail在CNS16190中找出這個控制措施到下一個控制措施之間的詳細內容。\
#                  輸出格式為: 控制措施{row['provision']}: 控制措施的詳細內容")
result_CNS16190 = rag_chain_CNS16190.invoke(f"{query}\n\n根據控制措施{row['provision']}和{row['detail']}\
                                            在CNS16190中找出這個控制措施到下一個控制措施之間的所有內容。\
                                            輸出格式為: 控制措施{row['provision']}: 控制措施的內容")

print(result_CNS16190)


5.5 安全通訊
控制措施5.5-
1：消費者IoT裝置應使用最佳實務密碼學安全通訊。安全控制措施之適切性及最
佳實務密碼學的使用，取決於許多因素，包括使用全景。隨著安全不斷演進，很難
提供關於密碼學或其他安全措施之規定性建議，而無此種建議很快過時的風險。
控制措施5.5-
2：消費者IoT裝置宜使用經審查或經評估之實作，交付網路功能性及安全功能性
，特別是密碼學領域。
審查及評估可能涉及獨立之內部或外部個體。
例 
1.開發及測試社群中散佈之軟體程式館、經驗證軟體模組，以及硬體設備加密服務提供者(諸如安
全元件及受信任執行環境)係全部經審查或評估。
控制措施5.5-3：密碼演算法及基元宜為可更新。
備考1. 此亦稱為〝密碼敏捷性〞。
對無法更新之裝置，重要的是裝置之預期壽命不超過裝置所使用密碼演算法的建議使用壽命(包括
金鑰長度)。
控制措施5.5-
4：宜僅於鑑別介面後，方可經由網路介面於初始化狀態存取裝置功能性。
備考2. 
於使用案例上功能性可能差異很大，且可能包含某些內容，包括存取個人資料及裝置致動器。
有些裝置可提供公開之開放資料，例：於WebofThings[18]中。此等裝置無需鑑別即可存取，從而
對所有人開放存取。
裝置可經由網路服務中之脆弱性遭受危害。適切之鑑別機制可防止未經授權的存取，且有助於裝
置中之縱深防禦。
控制措施5.5-
5：容許經由網路介面於組態中之安全相關變更的裝置功能性，僅應於鑑別後存取

5. 消費者IoT裝置之網宇安全控制措施
5.1 禁止使用通用預設通行碼
控制措施5.1-
1：使用通行碼且處於原廠預設值外之任何狀態時，所有消費者IoT裝置通行碼應
為每裝置唯一或由使用者所定義。
備考：有許多用以執行鑑別之機制，且通行碼並非用於對裝置鑑別使用者的唯一機制。然而若使
用，則建議依NIST Special Publication 800-
63B[3]，依循通行碼之最佳實務作法。對機器對機器之鑑別，使用通行碼通常不適切。
許多消費者IoT裝置於銷售時，具通用預設使用者名稱及通行碼(諸如〝admin,admin〞)，用於使用
者介面直至網路協定。繼續使用通用預設值係IoT中許多安全問題之根源[17]，需停止該實務作法
。上述控制措施可藉由使用每裝置唯一之預先安裝通行碼及/或藉由要求使用者選擇，依循最佳實
務作法的通行碼作為初始化

In [43]:
# rag_chain_TS103701.invoke(f"{query}\n\n根據控制措施和detail在根據控制措施和detail在ETSI TS 103 701中\
#                           找出The test group addresses the provision {row['provision']}.內容。\
#                  HINT: 控制措施英文是provision")
result_TS103701 = rag_chain_TS103701.invoke(f"{query}\n\n在ETSI TS 103 701中找出{row['provision']}Test group{row['provision']}到下一個Test group之間的內容，\
                          HINT: 控制措施就是provision。\
                          輸出格式為: Test group{row['provision']}內容: ")
# rag_chain_TS103701.invoke(f"在ETSI TS 103 701中\
#                           找出The test group addresses the provision 5.3-14.內容。")
print(result_TS103701)

ETSI ETSI TS 103 701 V1.1.1 (2021 -08) 53  
• for all critical security parameters the "Generation Mechan ism" ensures that the critical security parameters are 
unique per device and produced with a mechanism that reduces the risk of automated attacks against classes of 
devices. 
The verdict FAIL is assigned otherwise. 
5.5 TSO 5.5: Communicate securely 
5.5.1 Test group 5.5-1 
5.5.1.0 Test group objective 
The test group addresses the provision 5.5-1. 
According to ETSI TS 103 645 [1]/ETSI EN 303 645 [2] best practice cryptography is defined as cryptography that is 
suitable for the corresponding use case and has no indication of a feasible attack with current readily available techniques. 
The objective of this test group is to assess, firstly, whether the cryptographic methods provide the security guarantees 
that are necessary for the use case of the communication and, secondly, whether the cryptographic methods are not known to be vulnerable to a feasible attack.

ETSI ETSI TS 1

In [62]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Define the prompt template for generating the test scenario
prompt_testScenario = ChatPromptTemplate.from_template(
    "你是一個資安專家，使用以下控制措施與測試情境回答問題，如果不知道答案就說不知道。"
    "\n 問題：{question}"
    "\n答案："
)

# Create the test scenario chain
model_parser = llm | StrOutputParser()
rag_chain_testScenario = RunnableSequence(
    {"question": RunnablePassthrough()}  
    | prompt_testScenario 
    | model_parser
)


In [68]:
test_scenario_result = rag_chain_testScenario.invoke(f"{query}\n\n根據以下2點生成Test Scenario來驗證測試者驗證{row['answer']}的內容是否真的符合控制措施detail要求，\
                               1.控制措施: {result_CNS16190.content}，\
                               2.相關Test Scenario: {result_TS103701.content}。\
                                HINT: 可參考 回答範例{test_data} 來生成測試流程(Test Scenario)。\
                                除了生成的Test Scenario外，也必須包含兩個問題讓測試工程師檢驗是否符合上述兩點內容要求。\
                                !!請用繁體中文回答!!")
# rag_chain_testScenario.invoke(f"{query}\n\n根據以下2點生成Test Scenario來驗證測試者驗證answer的內容是否真的符合控制措施detail要求，\
#                                1.控制措施: {result_CNS16190}，\
#                                2.相關Test Scenario: {result_TS103701}。\
#                                 HINT: 可參考 回答範例 來生成測試流程(Test Scenario)。")
print(test_scenario_result)

Test Scenario：
1. 透過SSL/TLS v1.2以上的安全加密方式，測試D-Link網站所有通訊是否都是加密的，包含FW檔案的更新作業。
2. 檢查Cipher是否符合FIPS 14--3的安全強度模組，確保使用的加密方式達到最高等級。
3. 檢查D-Link是否隨時關注最新的安全通報，並且及時更新現行加密模組以解決可能的漏洞。

問題1：D-Link網站所有通訊是否都是在全程TLS v1.2以上的安全加密方式進行？
問題2：D-Link網站的FW檔案更新作業是否都是在全程TLS v1.2以上的安全加密方式進行？


In [69]:
# Define the prompt template for generating the test scenario
prompt_reason = ChatPromptTemplate.from_template(
    "你是一個資安專家，使用以下控制措施與測試情境回答問題，如果不知道答案就說不知道。"
    "\n 問題：{question}"
    "\n答案："
)

# Create the test scenario chain
model_parser = llm | StrOutputParser()
rag_chain_reasonResult = RunnableSequence(
    {"question": RunnablePassthrough()}  
    | prompt_reason 
    | model_parser
)

In [72]:
reson_result = rag_chain_reasonResult.invoke(f"{query}\n\n根據以下3點內容生成初步判定結果，驗證測試者驗證{row['answer']}的內容是否真的符合以下內容要求，\
                               1.控制措施: {result_CNS16190.content}，\
                               2.相關Test Scenario: {result_TS103701.content}，\
                               3.根據CNS16190、TS103701和answer所生成的測試情境: {test_scenario_result}。\
                               HINT: 回復方式請以json格式回答，欄位分別是: reason(初判結果原因)、result(初判結果: PASS FAIL)\
                                !!請用繁體中文回答!!")
# rag_chain_testScenario.invoke(f"{query}\n\n根據以下2點生成Test Scenario來驗證測試者驗證answer的內容是否真的符合控制措施detail要求，\
#                                1.控制措施: {result_CNS16190}，\
#                                2.相關Test Scenario: {result_TS103701}。\
#                                 HINT: 可參考 回答範例 來生成測試流程(Test Scenario)。")
print(reson_result)

{
    "reason": "根據測試情境中的要求，透過SSL/TLS v1.2以上的安全加密方式測試D-Link網站所有通訊，包含FW檔案的更新作業，確保全程使用安全加密方式進行通訊。",
    "result": "PASS"
}


### Batch test

In [198]:
queries_CNS16190=[]
queries_TS103701 = []
for row in range(len(df) - 3):
    row = df.iloc[row + 3]
    condition = (
        f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
    )
    # print(row)

    # query = f"{condition}\n\n根據控制措施和detail在ETSI TS 103 701中找出與控制措施最相關Test group，並生成測試流程來供測試者驗證answer 是否真的符合控制措施detail要求。"
    query_CNS16190 = f"{condition}\n\n根據控制措施{row['provision']}和{row['detail']}\
                                            在CNS16190中找出這個控制措施到下一個控制措施之間的所有內容。\
                                            輸出格式為: 控制措施{row['provision']}: 控制措施的內容"
    query_TS103701 = f"{condition}\n\n在ETSI TS 103 701中找出{row['provision']}Test group{row['provision']}到下一個Test group之間的內容，\
                        HINT: 控制措施就是provision。\
                        輸出格式為: Test group{row['provision']}內容: "
    

    queries_CNS16190.append(query_CNS16190)
    queries_TS103701.append(query_TS103701)


In [189]:
result_CNS16190 = rag_chain_CNS16190.batch(queries_CNS16190)
result_TS103701 = rag_chain_TS103701.batch(queries_TS103701)


控制措施5.3-
14：對無法更新其軟體之受限制裝置，製造者宜以對使用者清晰透通的可存取方
式，公布未更新軟體之理由闡述、硬體更換支援的期間及方法，以及定義之支援期
間。
控制措施5.3-15：對無法更新軟體之受限制裝置，產品宜為可隔離且硬體可更換。
於某些情況下，裝置無法修補程式。對受限制裝置，需備妥更換計畫並清楚傳達予消費者。此計
畫通常詳細說明何時需更換技術，以及適用時，何時結束對硬體及軟體之支援等的排程。
控制措施5.3-
16：消費者IoT裝置之型號名稱應藉由於裝置上加標籤，或經由實體介面可清晰辨
識。
此通常經由邏輯介面與裝置通訊所執行，但其亦可為UI之一部分。
例 
13.裝置具報告型號名稱之HTTP(或適切時為HTTPS)API(於使用者鑑別後)。通常要求瞭解裝置之特定
名稱，以核對定義的軟體更新支援期間或軟體更新的可用性。
5.4 安全儲存敏感性安全參數
控制措施5.4-1：持久性儲存體中之敏感性安全參數應由裝置安全儲存。
安全儲存機制可用以保護敏感性安全參數。適切之機制包括由受信任執行環境(TEE)所提供的機制
，以及與硬體、安全元件(SE)或專用安全組件(DSC)相關聯之加密儲存，並於UICC(依ETSITR121905[
29]、ETSITS102221[25])上/於嵌入式UICC(依GSMASGP.22TechnicalSpecificationv2.2.1[26])上運行的軟
體之處理能力。
備考：此控制措施適用於持久性儲存體，但製造者亦可對記憶體中之敏感性安全參數實作類似作
法。
例 1.授權及接取許可之無線電頻率(例：LTE-m細胞式接取)所涉及的根金鑰儲存於UICC中。
例 2.使用受信任執行環境(TEE)儲存並存取敏感性安全參數之遠端控制門鎖。
例 3.無線恆溫器將無線網路之信符儲存於防竄改微控制器中，而非於外部快閃儲存體中。

中華民國國家標準 CNS
消費者物聯網之網宇安全：基準要求事項 Cybersecurity for consumer 
internet of things: Baseline requirements
CNS16190:2023
4. 報告實作
本標準中控制措施之實作，係藉由風險評鑑及威脅建模(諸如CNS27005[27]及STRIDE威脅模型[28])
所告知；此係由裝置製造者及/或其他相關個體所執

In [190]:
queries_test_scenario = []
i = 0
for row in range(len(df) - 3):
    row = df.iloc[row + 3]
    condition = (
        f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
    )
    # 自己查找的CNS16190資料
    query_test_scenario = f"{condition}\n\n根據以下2點生成Test Scenario來驗證測試者驗證{row['answer']}的內容是否真的符合控制措施detail要求，\
                               1.控制措施: {result_CNS16190[i].content}，\
                               2.相關Test Scenario: {result_TS103701[i].content}。\
                                HINT: 可參考 回答範例{test_data} 來生成測試流程(Test Scenario)。\
                                除了生成的Test Scenario外，也必須包含兩個問題讓測試工程師檢驗是否符合上述兩點內容要求。\
                                !!請用繁體中文回答!!"
    # query_test_scenario = f"{condition}\n\n根據以下2點生成Test Scenario來驗證測試者驗證{row['answer']}的內容是否真的符合控制措施detail要求，\
    #                            1.控制措施: {row['provision']}{row['detail']}，\
    #                            2.相關Test Scenario: {result_TS103701[i].content}。\
    #                             HINT: 可參考 回答範例{test_data} 來生成測試流程(Test Scenario)。\
    #                             除了生成的Test Scenario外，也必須包含兩個問題讓測試工程師檢驗是否符合上述兩點內容要求。\
    #                             !!請用繁體中文回答!!"
    queries_test_scenario.append(query_test_scenario)
    i+=1
    


In [191]:
result_testScenario = rag_chain_testScenario.batch(queries_test_scenario)
result_testScenario


['生成的Test Scenario: \n1. 測試項目：測試製造商漏洞揭露政策是否符合要求\n   - 測試步驟：使用安全研究人員身份向製造商報告一個安全漏洞，觀察是否能夠透過製造商公開的漏洞揭露政策進行報告。同時觀察製造商是否及時更新漏洞狀態。\n   - 預期結果：製造商應當能夠清楚地提供報告安全漏洞的聯絡資訊，並且在收到初始確認後及時更新漏洞狀態，直到問題得到解決。\n\n2. 測試項目：測試裝置身份驗證資料在傳輸過程中的加密保護\n   - 測試步驟：模擬使用者向裝置進行身份驗證的過程，觀察身份驗證資料在傳輸過程中是否進行加密保護。使用 HTTPS TLS v1.2 或更新的連線進行測試。\n   - 預期結果：身份驗證資料在傳輸過程中應當經過加密保護，以確保密碼不會被簡單地讀取或洩漏。\n\n問題1：製造商如何組織漏洞揭露政策？是否公開了報告問題的聯絡資訊和相關時間表資訊？\n問題2：身份驗證資料在傳輸過程中如何進行加密保護？',
 '根據以上的控制措施及測試情境，生成測試流程如下：\n\nTest Scenario: \n根據 ETSI TS 103 645 [1]/ETSI EN 303 645 [2] 的要求，驗證消費者物聯網設備的型號名稱是否能夠通過設備上的標籤或實體介面清晰可識別。請驗證此產品的型號標籤是否包含相關的生產編號資訊。\n\n問題1：\n根據控制措施5.1-1的要求，當使用密碼且非出廠預設值時，請說明密碼是如何建立的。\n\n問題2：\n根據控制措施5.2-1的規定，請說明製造商組織的漏洞揭露政策，並確認是否公開了報告問題的聯絡資訊以及報告問題解決進度的時程表資訊。\n\n以上是根據控制措施及測試情境生成的測試流程，請在測試過程中確保符合相關要求並回答上述問題。',
 '根據上述控制措施5.4-1，可以生成以下兩個測試流程(Test Scenario)來驗證所有關鍵機敏資訊都以AES 128加密演算法處理存儲在機器的flash中：\n\n1. 測試流程1：使用模擬敏感安全參數的數據，確認這些數據在存儲時是否經過AES 128加密處理。透過讀取flash中的數據，並進行解密驗證，確保數據是安全存儲的。\n\n2. 測試流程2：模擬未經加密的敏感安全參數存儲在flash中，然後嘗試從該flash中讀取這些數據。確保系統能夠檢測到未加密

In [199]:
i = 0
queries_reason_result = []
for row in range(len(df) - 3):
    row = df.iloc[row + 3]
    condition = (
        f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
    )
    # 用自己生成的Test Scenario
    # query_reson_result = f"{query}\n\n根據以下3點內容生成初步判定結果，驗證測試者驗證{row['answer']}的內容是否真的符合以下內容要求，\
    #                            1.控制措施: {result_CNS16190[i].content}，\
    #                            2.相關Test Scenario: {result_TS103701[i].content}，\
    #                            3.根據CNS16190、TS103701和answer所生成的測試情境: {result_testScenario[i]}。\
    #                            輸出的回覆請務必遵守以下限制: 請將問題1 & 問題2 彙整成單一reason(初判結果原因)、result(初判結果: PASS FAIL)\
    #                             !!每一個控制措施只能輸出一個reason和result，如果超過一個只保留第一個!! \
    #                             回復方式請以json格式回答，欄位分別是: reason(初判結果原因)、result(初判結果: PASS FAIL)。\
    #                             !!請用繁體中文回答!!"
    # 用廠商提供的Test Scenario
    query_reson_result = f"{query}\n\n根據以下2點內容生成初步判定結果，驗證測試者驗證{row['answer']}的內容是否真的符合以下內容要求，\
                               1.控制措施: {row['provision']}{row['detail']}，\
                               2.根據CNS16190、TS103701和answer所生成的測試情境: {row['Test Scenario']}。\
                               輸出的回覆請務必遵守以下限制: 請將問題1 & 問題2 彙整成單一reason(初判結果原因)、result(初判結果: PASS FAIL)\
                                !!每一個控制措施只能輸出一個reason和result，如果超過一個只保留第一個!! \
                                回復方式請以json格式回答，欄位分別是: reason(初判結果原因)、result(初判結果: PASS FAIL)。\
                                !!請用繁體中文回答!!"
    
    queries_reason_result.append(query_reson_result)
    i +=1

In [200]:
result_reasonResult = rag_chain_reasonResult.batch(queries_reason_result)
result_reasonResult


['{\n  "reason": "客戶可以在產品網頁上找到定義的支援期限資訊。",\n  "result": "PASS"\n}',
 '{\n  "reason": "根據測試情境中提到的控制措施5.3-16，消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。驗證測試者驗證此產品的型號標籤上有相關的生產編號資訊。",\n  "result": "PASS"\n}',
 '{\n    "reason": "設備確保敏感安全參數安全儲存的方法可能包括使用安全、可信任的儲存機制來保護敏感安全參數，並且遵循相關的標準和規範要求。",\n    "result": "PASS"\n}',
 '{\n    "reason": "根據CNS16190、TS103701和answer所生成的測試情境，逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼，這是不良做法，可能導致安全風險。",\n    "result": "FAIL"\n}',
 '{\n    "reason": "根據CNS16190、TS103701和answer所生成的測試情境，驗證D-Link Cloud管理通訊使用每個設備唯一的設備證書以及配置獨特的關鍵安全參數，有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。",\n    "result": "PASS"\n}',
 '{\n    "reason": "D-Link網站的通訊都是在全程TLS V1.2以上的安全加密方式進行，並採用最高等級的Cipher符合FIPS 14--3的安全強度模組進行，並且隨時注意最新安全通報並更新加密模組，符合消費者物聯網設備應使用最佳實踐加密技術進行安全通訊的要求。",\n    "result": "PASS"\n}',
 '{\n    "reason": "D-Link 網站的通訊和FW檔案更新作業皆以TLS V1.2以上的安全加密方式進行，並採用最高等級的Cipher 符合FIPS 140-3的安全強度模組，並隨時注意最新的安全通報並更新加密模組，符合控制措施5.5-5的要求。",\n    "result": "PASS"\n}',
 '{\n    "reason": "D-Link網站的通訊都是在全程TLS V1.2以上的安全加密方式進行，並採用最高等級的Ci

In [172]:
import json
# a = json.loads(result_reasonResult[5])
# print(a['reason'])
# json_result = json.loads(result_reasonResult[15])

print(result_reasonResult[15])
# result_reasonResult[2]


{
  "reason": "根據控制措施5.1-1，所有消費者物聯網裝置密碼應至少包含長度為8位元，且包含大小寫字母、數字和特殊符號等多種元素。密碼應該由使用者自行創建，不應使用出廠預設值。",
  "result": "PASS"
}


In [201]:
import json
reason_list = []
result_list = []
i = 0
for result in result_reasonResult:
    json_result = json.loads(result)

    reason_list.append(json_result['reason'])
    result_list.append(json_result['result'])
    i+=1

# 自己生成的Test Scenario
# data = {
#     "provision": df["provision"].iloc[3:].tolist(),
#     "detail": result_CNS16190,
#     "status": df["status"].iloc[3:].tolist(),
#     "support": df["support"].iloc[3:].tolist(),
#     "Test Scenario": result_testScenario,
#     "answer": df["answer"].iloc[3:].tolist(),
#     "reason": reason_list,
#     "result": result_list,
# }

# 廠商提供的Test Scenario
data = {
    "provision": df["provision"].iloc[3:].tolist(),
    "detail": df["detail"].iloc[3:].tolist(),
    "status": df["status"].iloc[3:].tolist(),
    "support": df["support"].iloc[3:].tolist(),
    "Test Scenario": df["Test Scenario"].iloc[3:].tolist(),
    "answer": df["answer"].iloc[3:].tolist(),
    "reason": reason_list,
    "result": result_list,
}
# provision	status	support	detail	question(Test Ccenario會依據產品類型調整)	answer	reason	result

result_df = pd.DataFrame(data)
result_df

,provision,detail,status,support,Test Scenario,answer,reason,result
0,5.3-13,製造商應以對使用者易於理解、透明的方式公開定義的支援期限。,M,NaN,消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。,客戶可以在產品網頁上找到定義的支援期限資訊。,PASS
1,5.3-16,消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。,M,NaN,問題：1. 使用者如何驗證設備型號名稱（透過設備上的標籤或透過實體介面）？,此產品的型號標籤上有相關的生產編號資訊。,根據測試情境中提到的控制措施5.3-16，消費者物聯網設備的型號名稱應通過設備上的標籤或通過...,PASS
2,5.4-1,持久存儲中的敏感安全參數應由設備安全的存儲。,M,NaN,安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉...,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。,設備確保敏感安全參數安全儲存的方法可能包括使用安全、可信任的儲存機制來保護敏感安全參數，並且...,PASS
3,5.4-3,關鍵安全參數不得硬編碼於使用設備軟體原始碼中。,M,NaN,設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證...,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...,根據CNS16190、TS103701和answer所生成的測試情境，逆向工程可以輕鬆發現憑...,FAIL
4,5.4-4,用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個...,M,NaN,為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果...,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\n管理...,根據CNS16190、TS103701和answer所生成的測試情境，驗證D-Link Cl...,PASS
5,5.5-1,消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。,M,NaN,安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,D-Link網站的通訊都是在全程TLS V1.2以上的安全加密方式進行，並採用最高等級的Ci...,PASS
6,5.5-5,設備功能只有在認證存取之後允許透過網路介面進行配置安全相關修改。例外情況是設備所依賴的網路服...,M,NaN,注意：例外的協定包括 ARP、DHCP、DNS、ICMP 和 NTP。範例：與安全性相關的變...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,D-Link 網站的通訊和FW檔案更新作業皆以TLS V1.2以上的安全加密方式進行，並採用...,PASS
7,5.5-7,消費者物聯網設備應保護透過遠端存取網路介面的關鍵安全參數的機密性。,M,NaN,可以使用加密的通訊通道或有效負載加密來實現機密性保護。注意：加密保護通常是使用至少與傳輸的密...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,D-Link網站的通訊都是在全程TLS V1.2以上的安全加密方式進行，並採用最高等級的Ci...,PASS
8,5.5-8,製造商應遵循與設備相關的關鍵安全參數的安全管理流程。,M,NaN,注意：對關鍵安全參數使用開放的、同儕審查的標準（通常稱為「金鑰管理」）受到強烈鼓勵。問題：當...,按照D-Link的秘鑰管理流程生成、存儲和傳輸安全參數。 安全參數隨機生成后，寫入到設備中，...,製造商確實遵循安全管理流程，包括使用開放的、同儕審查的金鑰管理標準，並且將關鍵安全參數與設備...,PASS
9,5.6-1,應禁用所有未使用的網路和邏輯介面。,M,NaN,範例：預設情況下，無法從 WAN 存取應從 LAN 存取的管理 UI。注意：邏輯介面 (LI...,有一個 LAN 的乙太網路、2.4G 和 5G Wi-Fi 接口，需要在設備使用過程中運行，...,根據測試情境描述，驗證者確認只有LAN的乙太網路、2.4G和5G Wi-Fi接口被使用，沒有...,PASS


In [202]:
result_df.to_csv("A1+A2(廠商提供Test Scenario).csv", index=False, encoding="utf-8-sig")